In [9]:
from collections import defaultdict
from datetime import datetime
import json
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm

sys.path.append(os.path.abspath(".."))

from config import (
    FILTERED_DATA_PATH,
    TRAIN_DATA_PATH,
    VAL_DATA_PATH,
    TEST_DATA_PATH,
    METADATA_PATH,
    EMBEDDINGS_TITLE_PATH,
    EMBEDDINGS_DESCR_PATH,
    EMBEDDINGS_COMBI_PATH,
    RANDOM_STATE,
    N_RECOMMENDATIONS,
    N_EPOCHS,
    RECOMMENDATIONS_PATH,
    TRANSFORMED_DATA_PATH,
    SPLIT_DATE_TRAIN_VAL,
    SPLIT_DATE_VAL_TEST,
)
import utils.utils as utils

In [63]:
# importing training data
train_df = pd.read_parquet("../" + TRAIN_DATA_PATH)

# importing validation data
val_df = pd.read_parquet("../" + VAL_DATA_PATH)

# importing test data
test_df = pd.read_parquet("../" + TEST_DATA_PATH)

# importing metadata
meta_df = pd.read_parquet("../" + METADATA_PATH)

# importing the transformed data
transformed_df = pd.read_parquet("../" + TRANSFORMED_DATA_PATH)

# importing the filtered data
filtered_df = pd.read_parquet("../" + FILTERED_DATA_PATH)

# importing embeddings
title_embeddings = pd.read_parquet("../" + EMBEDDINGS_TITLE_PATH)

In [64]:
train_w_meta = train_df.merge(meta_df, how="left", on="prd_number")

In [66]:
val_df

,user_id,prd_number,completion_rate
0,00018423377c14104870be7deda65daaa9dbc7b97f89a8...,11032400013,0.973418
1,00018423377c14104870be7deda65daaa9dbc7b97f89a8...,11032400014,0.634413
3,00018423377c14104870be7deda65daaa9dbc7b97f89a8...,11032415442,0.950176
4,00018423377c14104870be7deda65daaa9dbc7b97f89a8...,11032419011,0.975945
5,00018423377c14104870be7deda65daaa9dbc7b97f89a8...,11032421433,1.000000
...,...,...,...
2365573,fffe6e1be686be369dfe0943282e377f7d1a51c0eae35a...,13652450206,1.000000
2365574,fffe6e1be686be369dfe0943282e377f7d1a51c0eae35a...,13652450207,0.996209
2365575,fffe6e1be686be369dfe0943282e377f7d1a51c0eae35a...,13652450208,0.429653
2365584,fffe6e1be686be369dfe0943282e377f7d1a51c0eae35a...,15452448044,1.000000


In [71]:
cf_scores = {"episode5411": -0.45, "episode4234": 0.03, "episode9724": 0.75}
cb_scores = {"episode5411": 0.22, "episode4235": 0.03, "episode9725": 0.75}

# Define the weight lambda
_lambda = 0.7

# Initialize the combined dictionary
combined_scores = {}

# Handle keys that are common to both dictionaries
for key in cf_scores.keys() & cb_scores.keys():  # Intersection of keys
    combined_scores[key] = _lambda * cf_scores[key] + (1 - _lambda) * cb_scores[key]

# Handle keys that are only in cf_scores
for key in cf_scores.keys() - cb_scores.keys():  # Keys unique to cf_scores
    combined_scores[key] = cf_scores[key]

# Handle keys that are only in cb_scores
for key in cb_scores.keys() - cf_scores.keys():  # Keys unique to cb_scores
    combined_scores[key] = cb_scores[key]

print(combined_scores)

{'episode5411': -0.249, 'episode9724': 0.75, 'episode4234': 0.03, 'episode9725': 0.75, 'episode4235': 0.03}
